# Activo la libreria de drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Verificacion tecnica del sistema.



In [2]:
from platform import python_version
import tensorflow as tf

print("Version de python: " + python_version())
print("Version de tensorflow: " + tf.__version__)

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

nombre_gpu = tf.test.gpu_device_name()
if nombre_gpu != '/device:GPU:0':
  raise SystemError('GPU no encontrada')
print('GPU encontrada: {}'.format(nombre_gpu))

Version de python: 3.7.10
Version de tensorflow: 2.5.0
Wed Jun  2 05:19:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    47W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                 

# Importamos librerias.

In [3]:
# import the libraries as shown below
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import keras.backend as K

In [4]:
# re-size all the images to this

train_path = '/content/drive/MyDrive/AgricultureVisión/Data/dataset-estadistica/train/rgb'
valid_path = '/content/drive/MyDrive/AgricultureVisión/Data/dataset-estadistica/test/rgb'

# Definir la estrategia de distribución
Creamos un objeto tipo MirroredStrategy. Esto manejará la distribución, y proporciona un gestor de contexto (tf.distribute.MirroredStrategy.scope) para construir su modelo dentro.

In [5]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [6]:
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 1


## Cuando se entrena un modelo con varias GPU, se puede utilizar la potencia de cálculo adicional de forma eficaz aumentando el tamaño del lote. En general, utilice el mayor tamaño de lote que se ajuste a la memoria de la GPU y ajuste la tasa de aprendizaje en consecuencia.

In [7]:
BUFFER_SIZE = 32

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

In [8]:
# useful for getting number of output classes
folders = glob(train_path + '/*')
num_imagenes = glob(train_path + '/*/*')

In [9]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [10]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (512, 512),
                                                 batch_size = BATCH_SIZE,
                                                 class_mode = 'categorical')

Found 1080 images belonging to 9 classes.


In [11]:
test_set = test_datagen.flow_from_directory(valid_path,
                                            target_size = (512, 512),
                                            batch_size = BATCH_SIZE,
                                            class_mode = 'categorical')

Found 450 images belonging to 9 classes.


In [12]:
def build_model(num_classes, img_size=512):
  with strategy.scope():
    input = tf.keras.layers.Input(shape=(img_size, img_size, 3))
    model = tf.keras.applications.NASNetLarge(include_top=False, input_tensor=input, weights="imagenet")

    # Rebuild top
    x = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    
    x = tf.keras.layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = tf.keras.layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    output = tf.keras.layers.Dense(num_classes, activation="softmax", name="pred")(x)

    
    for layer in model.layers[:-1]:
      layer.trainable = False
    
    #model.summary()
    # Compile
    model = tf.keras.Model(input, output, name="NASNetLarge")
    model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=["categorical_accuracy"])
    return model  

# Definir las llamadas de retorno
Los callbacks utilizados aquí son:



*   TensorBoard: Este callback escribe un registro para TensorBoard que permite visualizar las gráficas.
*   Model Checkpoint: Este callback guarda el modelo después de cada época.
*   Learning Rate Scheduler "Programador de la tasa de aprendizaje": Usando este callback, puedes programar la tasa de aprendizaje para que cambie después de cada epoch/lote.








In [13]:
# Define the checkpoint directory to store the checkpoints
import os

checkpoint_dir = '/content/drive/MyDrive/AgricultureVisión/models/callbacks/training_checkpoints_fd'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

# Function for decaying the learning rate.
# You can define any decay function you need.
def decay(epoch):
  if epoch < 3:
    return 1e-3
  elif epoch >= 3 and epoch < 7:
    return 1e-4
  else:
    return 1e-5

# Callback for printing the LR at the end of each epoch.
class PrintLR(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    print('\nTasa de aprendizaje para la época {} is {}'.format(epoch + 1,
                                                      model.optimizer.lr.numpy()))

In [14]:
callbacks_list = [
    tf.keras.callbacks.TensorBoard(log_dir='/content/drive/MyDrive/AgricultureVisión/models/callbacks/logs_fd'),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                       save_weights_only=True),
    tf.keras.callbacks.LearningRateScheduler(decay),
    PrintLR()
]

In [15]:
# useful for getting number of output classes
folders = glob(train_path + '/*')
num_imagenes = glob(train_path + '/*/*')

num_imagenes = glob(train_path + '/*/*')

num_classes = 9
num_images = len(num_imagenes)
#num_images = 70500

model = build_model(num_classes)

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [16]:
import time
import math

inicio = time.time()
history = model.fit(
    training_set,
    validation_data=test_set,
    epochs=50,
    steps_per_epoch=math.ceil(num_images/BATCH_SIZE))

fin = time.time()

Epoch 1/50
17/17 [==============================] - 295s 16s/step - loss: 1.7524 - categorical_accuracy: 0.4352 - val_loss: 1.8462 - val_categorical_accuracy: 0.4689
Epoch 2/50
17/17 [==============================] - 71s 4s/step - loss: 0.9312 - categorical_accuracy: 0.6917 - val_loss: 1.7321 - val_categorical_accuracy: 0.5244
Epoch 3/50
17/17 [==============================] - 71s 4s/step - loss: 0.6735 - categorical_accuracy: 0.7694 - val_loss: 1.6675 - val_categorical_accuracy: 0.5222
Epoch 4/50
17/17 [==============================] - 70s 4s/step - loss: 0.5649 - categorical_accuracy: 0.8167 - val_loss: 1.6147 - val_categorical_accuracy: 0.5444
Epoch 5/50
17/17 [==============================] - 70s 4s/step - loss: 0.4530 - categorical_accuracy: 0.8481 - val_loss: 1.5720 - val_categorical_accuracy: 0.5644
Epoch 6/50
17/17 [==============================] - 71s 4s/step - loss: 0.4071 - categorical_accuracy: 0.8704 - val_loss: 1.5308 - val_categorical_accuracy: 0.5511
Epoch 7/50
17/

In [17]:
print("Tiempo de entrenamiento gpu flow_from_directory strategy " + str((fin-inicio)/60) + " minutos")

Tiempo de entrenamiento gpu flow_from_directory strategy 62.50621162652969 minutos


In [18]:
def plotTraining(hist, epochs, typeData):
    
    if typeData=="loss":
        plt.figure(1,figsize=(10,5))
        yc=hist.history['loss']
        xc=range(epochs)
        plt.ylabel('Costo', fontsize=24)
        plt.plot(xc,yc,'-r',label='Costo entrenamiento')
    if typeData=="categorical_accuracy":
        plt.figure(2,figsize=(10,5))
        yc=hist.history['categorical_accuracy']
        for i in range(0, len(yc)):
            yc[i]=100*yc[i]
        xc=range(epochs)
        plt.ylabel('Accuracy (%)', fontsize=24)
        plt.plot(xc,yc,'-r',label='Accuracy entrenamiento')
    if typeData=="val_loss":
        plt.figure(1,figsize=(10,5))
        yc=hist.history['val_loss']
        xc=range(epochs)
        plt.ylabel('Costo', fontsize=24)
        plt.plot(xc,yc,'--b',label='Validación costo')
    if typeData=="val_categorical_accuracy":
        plt.figure(2,figsize=(10,5))
        yc=hist.history['val_categorical_accuracy']
        for i in range(0, len(yc)):
            yc[i]=100*yc[i]
        xc=range(epochs)
        plt.ylabel('Accuracy (%)', fontsize=24)
        plt.plot(xc,yc,'--b',label='Validación accuracy')
        

    plt.rc('xtick',labelsize=24)
    plt.rc('ytick',labelsize=24)
    plt.rc('legend', fontsize=18) 
    plt.legend()
    plt.xlabel('Numero de epocas',fontsize=24)
    plt.grid(True)

In [ ]:
epochs = 50
plotTraining(history,epochs,"loss")
plotTraining(history,epochs,"categorical_accuracy")
plotTraining(history,epochs,"val_loss")
plotTraining(history,epochs,"val_categorical_accuracy")

In [ ]:
from tensorflow.keras.models import load_model

model.save('/content/drive/MyDrive/AgricultureVisión/notebooks/mineria/Model/nasnet_50epo_from_directory_strategy.h5')